In [ ]:
!pip install -q "torch==2.2.1" tensorboard 

!pip install  -q --upgrade "transformers==4.40.0" "datasets==2.18.0" "accelerate==0.29.3" "evaluate==0.4.1" "bitsandbytes==0.43.1" "huggingface_hub==0.22.2" "trl==0.8.6" "peft==0.10.0" "optuna" "scikit-learn"

!huggingface-cli login --token "..."

In [ ]:
!pip install -q openai
!pip install -q accelerate evaluate
import torch
import evaluate
import re
import pandas as pd

In [ ]:
# resuse https://github.com/pkasela/DESIRE-ME/blob/main/src/model/utils.py
import logging
import os
import random

import numpy as np
import torch
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split

logger = logging.getLogger(__name__)


def seed_everything(seed: int):
    logger.info(f'Setting global random seed to {seed}')
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything(42)

In [ ]:
from openai import OpenAI


def gpt_generation_topic():
    client = OpenAI(api_key = "...")

    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      #model="gpt-3.5-turbo",
      messages = [
            #{
               # "role": "system",
               #"content": "Please come up with a topic, and based on the topic, generate a specific query and an ambiguous query. The output format should be: Topic: {topic}\n Specific query: {specific query}\n Ambiguous query: {ambiguous query}"
            #},
            {"role": "user", "content": "Please come up with a topic, and based on the topic, generate a specific query and an ambiguous query. The output format should be: Topic: {topic}\n Specific query: {specific query}\n Ambiguous query: {ambiguous query}"}],
      max_tokens=512,
      top_p=1,
      temperature=1.0
    )


    return completion.choices[0].message.content

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)


In [ ]:
def llama3_topic_generation():
    messages = [
        {"role": "user", "content": "Follow the instruction step by step. 1. Come up with 10 topics. 2. Based on the topics, generate 10 corresponding user information needs. 3. Based on the user information needs, generate 10 corresponding specific queries. 4. Based on the user information needs, generate 10 corresponding ambiguous queries. The output format should be Topic: {topic}\n User information need: {user information need}\n Specific query: {specific query}\n Ambiguous query: {ambiguous query}"},
    ]
    
    outputs = pipeline(
        messages,
        max_new_tokens=1024,
        temperature=1.0,
        do_sample=True
    
    )
    #print(outputs[0]["generated_text"][-1]['content'])
    return outputs[0]["generated_text"][-1]['content']

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

In [ ]:
def phi3_generation_topic():
     
    messages = [
        {"role": "user", "content": "Please come up with a topic, and based on the topic, generate a specific query and an ambiguous query. The output format should be: Topic: {topic}\n Specific query: {specific query}\n Ambiguous query: {ambiguous query}"},    
    ]

    
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
    )
    
    generation_args = {
        "max_new_tokens": 500,
        "return_full_text": False,
        "temperature": 1.0,
        "do_sample": True,
    }
    
    output = pipe(messages, **generation_args)
    #print(output[0]['generated_text'])
    return output[0]['generated_text']